In [ ]:
# ========================
# Install dependencies
# ========================
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q pandas

# ========================
# Imports
# ========================
import whisper
import pandas as pd
import requests
import os
from google.colab import files

# ========================
# Helper: Download video
# ========================
def download_video_robust(video_url, video_path):
    if os.path.exists(video_path):
        print("Video already downloaded.")
        return True
    try:
        response = requests.get(video_url, stream=True)
        response.raise_for_status()
        with open(video_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    except requests.exceptions.RequestException as e:
        print(f"Download failed: {e}")
        return False

# ========================
# Main transcription function
# ========================
def process_video_simple_to_df(video_url):
    file_name = video_url.split('/')[-1] or "video.mp4"
    video_path = f"./{file_name}"

    if not download_video_robust(video_url, video_path):
        return None

    model = whisper.load_model("small")
    result = model.transcribe(video_path, word_timestamps=False)

    rows = []
    for seg in result["segments"]:
        timestamp = f"[{seg['start']:.2f}s - {seg['end']:.2f}s]"
        rows.append({
            "file_name": file_name,
            "time_stamp": timestamp,
            "transcription": seg["text"].strip()
        })

    os.remove(video_path)
    return pd.DataFrame(rows)

# ========================
# Run transcription
# ========================
video_url = "https://ia601201.us.archive.org/0/items/CamelFilters/Camel%20filters.mp4"  # change this if needed
df = process_video_simple_to_df(video_url)

if df is not None:
    output_path = "/content/transcription.csv"
    df.to_csv(output_path, index=False)
    files.download(output_path)
else:
    print("Transcription failed.")
